# Using CV2 to extract images from multiple vieos & Get Metadata



#### This code:
    loads videos from your provided file path, 
    it reads the videos frames per second (fps)
    in this example i worked with 20second videos that had 30 fps and i needed only five images
    
    
Ater getting the 5 images, the os library, reads the videos' metadata of datetime created, and the piExif tool pastes it to rename every image in UTF-8 format ("%Y:%m:%d %H:%M:%S")

after looping through all the videos it then releases the videos 

    

In [1]:
import os
import cv2
import datetime
import piexif

# Load the videos, and get 
video_dir = "/Users/Documents/path"


video_files = [file for file in os.listdir(video_dir) if file.lower().endswith(".avi")]

for video_file in video_files:
    # Read the video file
    video_path = os.path.join(video_dir, video_file)
    cam = cv2.VideoCapture(video_path)

    # video properties
    fps = cam.get(cv2.CAP_PROP_FPS)

    # Initialize variables
    n = 0
    i = 0

    while True:
        ret, frame = cam.read()
        if not ret:
            break  

        # time in seconds
        time_in_seconds = n / fps

        # Run frames and save every 5 seconds
        if time_in_seconds >= 5 * i:
            # Get the video's date modified
            video_date = datetime.datetime.fromtimestamp(os.path.getmtime(video_path))
            formatted_date = video_date.strftime("%Y:%m:%d %H:%M:%S")
            image_filename = f"{formatted_date}_{video_file[:-4]}_frame_{i}.jpg"
            cv2.imwrite(image_filename, frame)

            # Update the image's EXIF metadata
            exif_dict = piexif.load(image_filename)
            new_date = video_date.strftime("%Y:%m:%d %H:%M:%S")  
            exif_dict['0th'][piexif.ImageIFD.DateTime] = new_date
            exif_bytes = piexif.dump(exif_dict)
            piexif.insert(exif_bytes, image_filename)

            i += 1

        n += 1

    # Release 
    cam.release()

print(f"Extracted images from {len(video_files)} videos.")

[mjpeg @ 0x7f793da42a80] overread 8


FileNotFoundError: [Errno 2] No such file or directory: '2023:12:04 16:29:26_IMG_1750_frame_2.jpg'

# USING piEXIF tool

this tool will help you manipulate image metadata 

For example list image metadata

In [ ]:
import piexif

image_path = "/images/pic.jpg"

# Load the EXIF data
exif_dict = piexif.load(image_path)

# Print all the metadata
print(f"Metadata for {image_path}:")
for ifd_name in exif_dict:
    print(f"IFD: {ifd_name}")
    if ifd_name == "thumbnail":
        continue
    for tag_name in exif_dict[ifd_name]:
        tag_value = exif_dict[ifd_name][tag_name]
        print(f"  {tag_name}: {tag_value}")

### Asuming you now have a long list of images and you want to loop the process and want to edit the general metadata data.

#### This code will help: 
            - copy Exif datetime exif_dict.get(306) which you don't need to worry about, i figured its hidden place
            - paste it on the general image *created* and *modified* datetime which is what you actually need instead of what piExif pastes it on 
          


In [ ]:
import os
from PIL import Image
from datetime import datetime

def copy_exif_and_update_timestamp(image_path):
    try:
        # Open the image
        img = Image.open(image_path)

        # Get the EXIF metadata
        exif_dict = img._getexif()

        # Extract relevant information (e.g., timestamp)
        timestamp_str = exif_dict.get(306)  # 306 corresponds to DateTimeOriginal tag

        # Parse the timestamp string into a datetime object
        timestamp_dt = datetime.strptime(timestamp_str, "%Y:%m:%d %H:%M:%S")

        # Convert datetime to timestamp (in seconds since the epoch)
        timestamp_int = int(timestamp_dt.timestamp())

        # Update the file timestamp
        os.utime(image_path, (timestamp_int, timestamp_int))

        print(f"Timestamp updated successfully for {image_path}")
    except Exception as e:
        print(f"Error: {e}")

# Specify the folder containing your images
folder_path = "/images/pic.jpg"

# Loop through all files in the folder
for filename in os.listdir(folder_path):
    if filename.lower().endswith(".jpg"):
        image_path = os.path.join(folder_path, filename)
        copy_exif_and_update_timestamp(image_path)